# Tratamento Profissional de Dados – Ocorrências e Reclamações (Limpeza, validação e otimização de dados, com relatório completo de decisões.)

In [33]:
import pandas as pd
import numpy as np
import unicodedata
import os

In [34]:
pd.set_option("display.max_columns", None)

In [35]:
# Função auxiliar: remover acentos
def strip_accents(s):
    if pd.isna(s):
        return s
    return ''.join(c for c in unicodedata.normalize('NFKD', str(s))
                   if not unicodedata.combining(c))


In [39]:
# 1️⃣ Carregar dados
path = "../raw_datasets/ocorrencias_reclamacoes.csv"
if not os.path.exists(path):
    raise FileNotFoundError(f"Ficheiro não encontrado: {path}")

df = pd.read_csv(path)

DECISOES = []
def registrar_decisao(txt):
    DECISOES.append("- " + txt)
    print("✔", txt)

In [ ]:

df_original = df.copy()    # cópia 100% fiel para referência
df_clean = df.copy()       # cópia sobre a qual vamos trabalhar
df = df_clean              # trabalhar sempre com df

In [53]:
# IDs de ocorrência que aparecem mais de uma vez
ids_repetidos = (
    df_clean.groupby('ocorrencia_id')['fração_origem']
    .nunique()
    .reset_index(name='num_frações')
)
duplicados = ids_repetidos[ids_repetidos['num_frações'] > 1]
print(f"Ocorrencias com o mesmo ID mas em frações diferentes: {len(duplicados)}")
display(duplicados.head(20))


Ocorrencias com o mesmo ID mas em frações diferentes: 0


,ocorrencia_id,num_frações


In [41]:
# 1. Corrigir nomes de colunas
ren = {c: c.strip() for c in df_clean.columns if c != c.strip()}
if ren:
    df_clean.rename(columns=ren, inplace=True)
    registrar_decisao(f"Nomes de colunas corrigidos: {ren}")

✔ Nomes de colunas corrigidos: {'    ocorrencia_id': 'ocorrencia_id'}


In [42]:
# 2. Duplicados
dups = df_clean.duplicated().sum()
if dups > 0:
    df_clean = df_clean.drop_duplicates()
    registrar_decisao(f"Removidos {dups} registos duplicados.")
else:
    registrar_decisao("Nenhum registo duplicado encontrado.")

✔ Removidos 26 registos duplicados.


In [43]:
# 3) Valores ausentes (relatório)
print("\nValores nulos (absoluto):")
print(df_clean.isna().sum())
print("\nValores nulos (%):")
print((df_clean.isna().sum()/len(df_clean)*100).round(2))


Valores nulos (absoluto):
ocorrencia_id          0
fração_origem          0
tipo                   0
descricao_completa    28
data_ocorrencia        0
gravidade              0
resolvido              0
acao_tomada           18
dtype: int64

Valores nulos (%):
ocorrencia_id         0.00
fração_origem         0.00
tipo                  0.00
descricao_completa    2.33
data_ocorrencia       0.00
gravidade             0.00
resolvido             0.00
acao_tomada           1.50
dtype: float64


In [44]:
# Decisão: 'resolvido' nulo -> "Investigar" (não assumimos 'Não')
if df_clean['resolvido'].isna().sum() > 0:
    df_clean['resolvido'] = df_clean['resolvido'].fillna('Investigar')
    registrar_decisao("Valores nulos em 'resolvido' preenchidos com 'Investigar' (possível erro/pendência).")


In [45]:
# 5. Padronização de formatos
# 5a) Normalizar 'tipo' (sem criar coluna auxiliar)
tipo_norm = (
    df_clean['tipo'].astype(str)
                     .str.strip()
                     .str.lower()
                     .apply(strip_accents)
)
mapeamento = {
    'ruido': 'Ruído', 'ruido.': 'Ruído', 'barulho': 'Ruído',
    'infiltracao': 'Infiltração', 'humidade': 'Infiltração',
    'vandalismo': 'Vandalismo', 'vandalism': 'Vandalismo',
    'limpeza': 'Limpeza', 'higiene': 'Limpeza',
    'seguranca': 'Segurança',
    'estacionamento': 'Estacionamento', 'parking': 'Estacionamento', 'garagem': 'Estacionamento'
}
df_clean['tipo'] = tipo_norm.replace(mapeamento)
registrar_decisao("Padronizada a coluna 'tipo' (acentos, maiúsculas e sinónimos).")


✔ Padronizada a coluna 'tipo' (acentos, maiúsculas e sinónimos).


/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2559/1031024117.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['tipo'] = tipo_norm.replace(mapeamento)


In [46]:
# 5b) Normalizar 'resolvido' para Sim/Não/Investigar
def norm_res(x):
    if pd.isna(x): return "Investigar"
    s = str(x).strip().lower()
    if s in {"sim","s","true","1"} or x is True: return "Sim"
    if s in {"não","nao","n","false","0"} or x is False: return "Não"
    return "Investigar"

df_clean['resolvido'] = df_clean['resolvido'].apply(norm_res).astype('category')
registrar_decisao("Normalizada a coluna 'resolvido' (Sim/Não/Investigar).")


✔ Normalizada a coluna 'resolvido' (Sim/Não/Investigar).


/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2559/977232012.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['resolvido'] = df_clean['resolvido'].apply(norm_res).astype('category')


In [47]:
# 5c) Limpar 'descricao_completa'
def limpar_descricao(txt):
    if pd.isna(txt): return txt
    t = str(txt).strip().lstrip('"\'' )
    letras = [c for c in t if c.isalpha()]
    if letras and sum(c.isupper() for c in letras) / len(letras) > 0.7:
        t = t.capitalize()
    return t

df_clean['descricao_completa'] = df_clean['descricao_completa'].apply(limpar_descricao)
registrar_decisao("Normalizada a coluna 'descricao_completa' (remoção de aspas iniciais e ajuste de maiúsculas).")


✔ Normalizada a coluna 'descricao_completa' (remoção de aspas iniciais e ajuste de maiúsculas).


/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2559/3992850040.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['descricao_completa'] = df_clean['descricao_completa'].apply(limpar_descricao)


In [48]:
# 5d) Padronizar 'acao_tomada' (Em analise → Em análise)
if 'acao_tomada' in df_clean.columns:
    df_clean['acao_tomada'] = (
        df_clean['acao_tomada']
        .astype(str)
        .str.strip()
        .str.replace('Em analise', 'Em análise', case=False, regex=False)
    )
    registrar_decisao("Padronizados valores de 'acao_tomada': 'Em analise' → 'Em análise'.")


✔ Padronizados valores de 'acao_tomada': 'Em analise' → 'Em análise'.


/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2559/3918207339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['acao_tomada'] = (


In [49]:
# 6. Conversão de datas e tipos
df_clean['data_ocorrencia'] = pd.to_datetime(df_clean['data_ocorrencia'], errors='coerce')

# Corrigir datas futuras
futuros = df_clean['data_ocorrencia'] > pd.Timestamp.now()
if futuros.any():
    df_clean.loc[futuros, 'data_ocorrencia'] = pd.Timestamp.now().normalize()
    registrar_decisao(f"{futuros.sum()} registos com data futura ajustados para hoje.")

# Remover hora
df_clean['data_ocorrencia'] = df_clean['data_ocorrencia'].dt.strftime('%Y-%m-%d')
registrar_decisao("Removido o horário de 'data_ocorrencia' (ficou apenas AAAA-MM-DD).")

# Categorização final
for c in ['fração_origem', 'tipo', 'resolvido']:
    df_clean[c] = df_clean[c].astype('category')


✔ Removido o horário de 'data_ocorrencia' (ficou apenas AAAA-MM-DD).


/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2559/323705884.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['data_ocorrencia'] = pd.to_datetime(df_clean['data_ocorrencia'], errors='coerce')
/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2559/323705884.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['data_ocorrencia'] = df_clean['data_ocorrencia'].dt.strftime('%Y-%m-%d')
/var/folders/zn/76llcf_53y1gktzpnrpfpcdr0000gn/T/ipykernel_2559/323705884.py:16: Setti

In [50]:
# 7. Outliers em gravidade
if 'gravidade' in df_clean.columns:
    outliers = (~df_clean['gravidade'].between(1,5)).sum()
    if outliers:
        df_clean.loc[df_clean['gravidade'] < 1, 'gravidade'] = 1
        df_clean.loc[df_clean['gravidade'] > 5, 'gravidade'] = 5
        registrar_decisao(f"Corrigidos {outliers} registos de 'gravidade' fora do intervalo 1–5.")
    else:
        registrar_decisao("Nenhum outlier de 'gravidade' encontrado.")


✔ Corrigidos 16 registos de 'gravidade' fora do intervalo 1–5.


In [51]:
# 8. Eliminar duplicados por ocorrencia_id, mantendo a linha mais completa
tmp = df_clean.copy()
tmp['__nn'] = tmp.notna().sum(axis=1)
tmp['__idx'] = np.arange(len(tmp))
tmp['__data_dt'] = pd.to_datetime(tmp['data_ocorrencia'], errors='coerce')
tmp = tmp.sort_values(['ocorrencia_id','__nn','__data_dt','__idx'])
df_clean = tmp.drop_duplicates(subset=['ocorrencia_id'], keep='last').drop(columns=['__nn','__idx','__data_dt'])
registrar_decisao("Removidos duplicados de 'ocorrencia_id', retendo registo mais completo.")


✔ Removidos duplicados de 'ocorrencia_id', retendo registo mais completo.


In [52]:
# 9. Otimização de memória
m0 = df_original.memory_usage(deep=True).sum()/1024**2
m1 = df_clean.memory_usage(deep=True).sum()/1024**2
registrar_decisao(f"Memória reduzida de {m0:.2f} MB para {m1:.2f} MB.")

✔ Memória reduzida de 0.67 MB para 0.54 MB.


In [54]:
# 10. Guardar resultados
out_dir = "../datasets_cleaned"
os.makedirs(out_dir, exist_ok=True)
clean_path = os.path.join(out_dir, "ocorrencias_cleaned_final.csv")
df_clean.to_csv(clean_path, index=False)
print(f"\n✔ Dataset limpo guardado em: {clean_path}")

with open(os.path.join(out_dir, "ocorrencias_decisoes.md"), "w", encoding="utf-8") as f:
    f.write("\n".join(DECISOES))
print("✔ Relatório de decisões salvo em: ../datasets_cleaned/ocorrencias_decisoes.md")


✔ Dataset limpo guardado em: ../datasets_cleaned/ocorrencias_cleaned_final.csv
✔ Relatório de decisões salvo em: ../datasets_cleaned/ocorrencias_decisoes.md
